In [1]:
# Modified 3rd party lib
# Install bio2byte-PeptideBuilder from GitHub if needed 

# !pip install git+https://github.com/Bio2Byte/bio2byte-peptidebuilder

In [2]:
import sys, os
import Bio.PDB, Bio.SeqUtils

import bio2byte.PeptideBuilder as PeptideBuilder
from bio2byte.PeptideBuilder import Geometry

In [3]:
class ResettingCycler:
    """ A cycler that starts from the same position every time invoked """
    def __init__(self, iterable):
        self.elements = iterable
        assert len(self.elements) > 1
    
    def __iter__(self):
        self.generator = (elem for elem in self.elements)
        return self
    
    def __next__(self):
        try:
            elem = next(self.generator)
        except StopIteration:
            self.generator = (elem for elem in self.elements)
            elem = next(self.generator)
        finally:
            return elem


def seq2name(sequence, prefix="./", suffix="", extension=".pdb"):
    outname = ""
    for resn in sequence:
        if resn.isupper():
            outname += resn
        else:
            r = Bio.SeqUtils.seq1(resn)
            assert r != 'X'
            outname += r.lower()
    return prefix + outname + suffix + extension

# Canonical amino acids
canonicalAA = [
    "Ala", "Arg", "Asn", "Asp", "Cys",
    "Gln", "Glu", "Gly", "His", "Ile",
    "Leu", "Lys", "Met", "Phe", "Pro",
    "Ser", "Thr", "Trp", "Tyr", "Val",
]
# Protomers of standard amino acids
protAA = [
    "HIP", "GLH", "ASH"
]
# Phosphorylated amino acids
phosAA = [
    "SEP", "TPO", "PTR",
    "S1P", "T1P", "Y1P",
]

# Enumerate peptides


#### Pentapetides

* $ 20 + 3 + 6 = 29 $
* Ace-Gly-Gly-**Xxx**-Gly-Gly-Nme


#### Hexapeptides

* $ 29^2 = 841 $
* Ace-Gly-Gly-**Xx1**-**Xx2**-Gly-Gly-Nme


#### Heptapeptides

* $ 29^2 = 841 $
* Ace-Gly-Gly-**Xx1**-Gly-**Xx2**-Gly-Gly-Nme


#### Octapeptides

* $ 29^2 = 841 $
* Ace-Gly-Gly-**Xx1**-Gly-Gly-**Xx2**-Gly-Gly-Nme


#### Nonapeptides

* $ 29^2 = 841 $
* Ace-Gly-Gly-**Xx1**-Gly-Gly-Gly-**Xx2**-Gly-Gly-Nme

# Generate list of all peptides

```python
allAA = [aa.upper() for aa in canonicalAA+protAA+phosAA]

with open("peptides.lst", 'w') as f:
    # Pentapeptides
    for resn in allAA:
        f.write("Gly-Gly-{0}-Gly-Gly\n".format(resn))
    
    # Hexapeptides
    for resn1 in allAA:
        for resn2 in allAA:
            f.write("Gly-Gly-{0}-{1}-Gly-Gly\n".format(resn1, resn2))
    
    # Heptapeptides
    for resn1 in allAA:
        for resn2 in allAA:
            f.write("Gly-Gly-{0}-Gly-{1}-Gly-Gly\n".format(resn1, resn2))
    
    # Octapeptides
    for resn1 in allAA:
        for resn2 in allAA:
            f.write("Gly-Gly-{0}-Gly-Gly-{1}-Gly-Gly\n".format(resn1, resn2))
    
    # Nonapeptides
    for resn1 in allAA:
        for resn2 in allAA:
            f.write("Gly-Gly-{0}-Gly-Gly-Gly-{1}-Gly-Gly\n".format(resn1, resn2))
```

# Build peptides

* Each peptide is build in four conformations
* Conformations alternate sheet/helix backbone dihedrals

In [4]:
h = (-60., -40.); s = (-135., 135.)

conformations = [
    ResettingCycler([s,s,h]),
    ResettingCycler([s,h,s]),
    ResettingCycler([h,h,s]),
    ResettingCycler([h,s,h])
]

with open("peptides.lst", 'r') as f:
    peptide_list = f.read().splitlines()
print(len(peptide_list))

In [6]:
if not os.path.isdir("built_peptides"):
    os.makedirs("built_peptides")

for i,peptide in enumerate(peptide_list):
    sequence = peptide.split('-')
    
    for j,confCycler in enumerate(conformations):
        
        structure = PeptideBuilder.initialize_ACE()    
        for resn, (phi,psi) in zip(sequence, confCycler):
            geo = Geometry.geometry(resn)
            geo.phi = phi
            geo.psi_im1 = psi
            PeptideBuilder.add_residue(structure, geo)
        PeptideBuilder.add_terminal_NME(structure)
    
        outname = seq2name(sequence, prefix="built_peptides/", suffix=".{:d}".format(j+1))
        out = Bio.PDB.PDBIO()
        out.set_structure(structure)
        out.save(outname)
    
    if i%10 == 0 and i > 0:
        print('.', end='')
        if i%250 == 0:
            print()